In [1]:
#GPU Reset if needed

from numba import cuda 
device = cuda.get_current_device()
device.reset()

In [2]:
# Parameters

base_directory = "i:/image_classify/" # base directory
image_size  = (224, 224) # preprocessing image size for model input to match with input_tensor in the model
classify_names = ["type1_male", "type2_male", "type3_male", "type4_male","type1_female", "type2_female", "type3_female", "type4_female"] # class names to be classified into (same as sub folder names with images in different classes)

# Description given in dataset link is incorrect, correct descriptions should be as below.
classify_desc = ["Male, mask is worn correctly, covers the nose and mouth", "Male, mask covers the mouth, but does not cover the nose", "Male, mask is on, but does not cover the nose or mouth.", "Male, no mask on the face","Female, mask is worn correctly, covers the nose and mouth", "Female, mask covers the mouth, but does not cover the nose", "Female, mask is on, but does not cover the nose or mouth.", "Female, no mask on the face"]

In [3]:
# Change to base directory

import os
os.chdir(base_directory)
print("base directory : " + os.getcwd())


base directory : i:\image_classify


In [4]:
import requests
import numpy as np
import cv2
import matplotlib.pyplot as plt

# Uncomment and use suitable code block from below two code blocks depending on a local file or URL for an image

In [5]:
### Load image from local file

# test_image = "test_img.jpg"

# image_src = cv2.imread(test_image)
# plt.imshow(image_src)


In [14]:
### Load image from a URL

test_image = "https://test_url"

img_from_url = requests.get(test_image)
arr = np.asarray(bytearray(img_from_url.content), dtype = np.uint8)
image_src = cv2.imdecode(arr, -1)
#plt.imshow(image_src)

In [7]:
# image file and preprocessing

image = cv2.resize(image_src,dsize=(image_size[0],image_size[1]), interpolation = cv2.INTER_CUBIC) 
np_image_data = np.asarray(image)

image = np.expand_dims(image, axis=0)

In [8]:
# Import Tensorflow and related libraries

import tensorflow

from tensorflow.keras.models import load_model

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input

print("TensorFlow version : " + tensorflow.__version__)

TensorFlow version : 2.9.0-dev20220102


In [9]:
# Display available GPUs

physical_devices = tensorflow.config.list_physical_devices('GPU')
for gpu in physical_devices:
    print("Name:", gpu.name, "  Type:", gpu.device_type)

Name: /physical_device:GPU:0   Type: GPU


In [10]:
# Enable Memory Growth

if physical_devices != []:
    tensorflow.config.experimental.set_memory_growth(physical_devices[0], True)
    physical_devices[0]

In [11]:
# load the saved model

saved_model = load_model('identify_final.h5')

# create a model with inpots and outputs to evaluate
input_tensor = Input((224, 224, 3))
model = Model(inputs=input_tensor, outputs=saved_model(input_tensor))
model._name = "verification"
model.summary()

Model: "verification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 identify (Functional)       (None, 8)                 53477182  
                                                                 
Total params: 53,477,182
Trainable params: 0
Non-trainable params: 53,477,182
_________________________________________________________________


In [12]:
predict_class = model.predict(image)
print("Preditcation array: " + str(predict_class) + "\n")

predict_class = np.argmax(predict_class[0])
print("Preditcation index: " + str(predict_class))

prediction = classify_names[predict_class]
prediction_desc = classify_desc[predict_class]

Preditcation array: [[6.6293523e-02 8.6844355e-01 2.7361808e-02 1.3680331e-02 4.1277823e-03
  1.8662220e-02 9.3854347e-04 4.9225119e-04]]

Preditcation index: 1


In [15]:
print(prediction)
print(prediction_desc)
#plt.imshow(image_src)

type2_male
Male, mask covers the mouth, but does not cover the nose
